In [10]:
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras.applications import vgg19
import tensorflow as tf
target_image_path='/Users/oliverzimmermann/Desktop/VisKomm3_A5/Berthold.Hannes.VK3.WiSe23.A5/BhfWerns.jpg'
style_reference_image_path='/Users/oliverzimmermann/Desktop/VisKomm3_A5/Berthold.Hannes.VK3.WiSe23.A5/Sternennacht.jpg' # Stil
width, height=load_img(target_image_path).size
img_height=400
img_width=int(width*img_height/height) #images anpassen 
print(img_width)

tf.compat.v1.disable_eager_execution()

533


In [11]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height,img_width))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=vgg19.preprocess_input(img)
    return img
def deprocess_image(x):
    x=x.astype('float64')
    x[:, :, 0] +=103.939
    x[:, :, 1] +=116.779
    x[:, :, 2] +=123.68
    x=x[:, :, ::-1]
    x=np.clip(x,0,255).astype('uint8')
    return x


In [12]:
from keras import backend as K
target_image=K.constant(preprocess_image(target_image_path))
style_reference_image=K.constant(preprocess_image(style_reference_image_path))
combination_image=K.placeholder((1, img_height, img_width, 3))
input_tensor=K.concatenate([target_image, style_reference_image,combination_image],axis=0)
model=vgg19.VGG19(input_tensor=input_tensor, weights='imagenet',include_top=False)
print('Modell feddich eingelesen')

2024-07-03 09:31:59.857336: W tensorflow/c/c_api.cc:304] Operation '{name:'block5_conv4_1/bias/Assign' id:1637 op device:{requested: '', assigned: ''} def:{{{node block5_conv4_1/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](block5_conv4_1/bias, block5_conv4_1/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Modell feddich eingelesen


In [13]:
def content_loss(base,combination):
    return K.sum(K.square(combination - base))
def gram_matrix(x):
    features=K.batch_flatten(K.permute_dimensions(x,(2,0,1)))
    gram=K.dot(features, K.transpose(features))
    return gram
def style_loss(style,combination):
    S=gram_matrix(style)
    C=gram_matrix(combination)
    channels=3
    size=img_height*img_width
    return K.sum(K.square(S-C))/(4.*(channels**2)*(size**2))


In [14]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width-1, :] - x[:, 1:, :img_width-1, :])  # Removed extra dimension from second operand
    
    b = K.square(
        x[:, :img_height - 1,:img_width-1, :] - x[:, :img_height -1, 1:, :])  # Adjusted slicing for width
    return K.sum(K.pow(a + b, 1.25))


In [15]:
outputs_dict=dict([(layer.name, layer.output) for layer in model.layers])
content_layer='block5_conv2'
style_layers=['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
total_variation_weight=1e-4
style_weight=1.
content_weight=0.025  #parametrisierbar zum ändern des bildes
loss=K.variable(0.)
layer_features=outputs_dict[content_layer]
target_image_features=layer_features[0,:,:,:]
combination_features=layer_features[2,:,:,:]
loss=loss+content_weight*content_loss(target_image_features,combination_features)
for layer_name in style_layers:
    layer_features=outputs_dict[layer_name]
    style_reference_features=layer_features[1,:,:,:]
    combination_features=layer_features[2,:,:,:]
    sl=style_loss(style_reference_features, combination_features)
    loss=loss+(style_weight/len(style_layers))*sl

loss=loss+total_variation_weight*total_variation_loss(combination_image)


In [16]:
grads=K.gradients(loss, combination_image)[0]
fetch_loss_and_grads=K.function([combination_image],[loss,grads])
class Evaluator(object):
    def __init__(self):
        self.loss_value=None
        self.grad_values=None #!
    def loss(self,x):
        assert self.loss_value is None
        x=x.reshape((1,img_height,img_width,3))
        outs=fetch_loss_and_grads([x])
        loss_value=outs[0]
        grad_values=outs[1].flatten().astype('float64')
        self.loss_value=loss_value
        self.grad_values=grad_values
        return self.loss_value
    def grads(self,x):
        assert self.loss_value is not None
        grad_values=np.copy(self.grad_values)
        self.loss_value=None
        self.grad_values=None
        return grad_values
evaluator=Evaluator()


In [17]:
from scipy.optimize import fmin_l_bfgs_b
from PIL import Image#from scipy.misc import imsave
import time
result_prefix='my_result'
iterations=20
x=preprocess_image(target_image_path)
x=x.flatten()

def save_img(img):
  pil_img=deprocess_image(np.copy(img))
  img = Image.fromarray(pil_img)
  img.save("neural_style_output_image.jpg")

for i in range(iterations):
    print('Starte Durchlauf',i)
    start_time=time.time()
    x,min_val,info=fmin_l_bfgs_b(evaluator.loss,x,fprime=evaluator.grads,maxfun=20)
    print('Wert Verlust:',min_val)
    img=x.copy().reshape((img_height,img_width,3))
    img=deprocess_image(img)
    fname=result_prefix + '_at_iteration_%d.png'%i
    save_img(img)

    print('Bild gespeichert als',fname)
    end_time=time.time()
    print('Durchlauf %d abgeschlossen in %ds'%(i,end_time-start_time))

Starte Durchlauf 0


2024-07-03 09:32:19.301113: W tensorflow/c/c_api.cc:304] Operation '{name:'Variable_1/Assign' id:1777 op device:{requested: '', assigned: ''} def:{{{node Variable_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](Variable_1, Variable_1/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Wert Verlust: 3311798800.0
Bild gespeichert als my_result_at_iteration_0.png
Durchlauf 0 abgeschlossen in 295s
Starte Durchlauf 1
Wert Verlust: 1426793300.0
Bild gespeichert als my_result_at_iteration_1.png
Durchlauf 1 abgeschlossen in 291s
Starte Durchlauf 2
Wert Verlust: 894618000.0
Bild gespeichert als my_result_at_iteration_2.png
Durchlauf 2 abgeschlossen in 290s
Starte Durchlauf 3
Wert Verlust: 658599700.0
Bild gespeichert als my_result_at_iteration_3.png
Durchlauf 3 abgeschlossen in 291s
Starte Durchlauf 4
Wert Verlust: 507613340.0
Bild gespeichert als my_result_at_iteration_4.png
Durchlauf 4 abgeschlossen in 277s
Starte Durchlauf 5


KeyboardInterrupt: 